In [1]:
from syllablecounter import load_model, get_data

In [2]:
def printmd(string, size = 3):
    from IPython.display import Markdown, display
    display(Markdown(f'<font size = {size}>{string}</font>'))
def green(string, deg = 1):
    return f'<span style = "background-color:hsl(125, {deg * 100}%, 50%);">{string}</span>'
def red(string, deg = 1):
    return f'<span style = "background-color:hsl(0, {deg * 100}%, 50%);">{string}</span>'
def ctext(string, deg = 0.5):
    deg = 2 * deg - 1
    if deg > 0:
        return green(string, deg)
    else:
        return red(string, -deg)

In [3]:
counter = load_model()
counter

SyllableCounter(
  (embed): Embedding(28, 64)
  (rnn): GRU(64, 128, num_layers=3, dropout=0.15, bidirectional=True)
  (drops): ModuleList(
    (0): Dropout(p=0.5, inplace=False)
    (1): Dropout(p=0.5, inplace=False)
  )
  (norms): ModuleList(
    (0): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lins): ModuleList(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out): Linear(in_features=256, out_features=1, bias=True)
)

In [4]:
import pandas as pd
df = pd.read_csv('data/gutsyls.tsv', sep = '\t')
df.head()

,words,syl_seq,syls
0,brigadiership,1000110001000,4
1,cauline,1001000,2
2,mousetrap,100001000,2
3,limina,100110,3
4,overlapping,11001001000,4


In [5]:
hard_words = {'words': [], 'syl_seq': [], 'syls': []}
for idx in range(1000):
    yhat = counter.predict(df['words'][idx], return_confidence = True, return_sequence = True)
    if yhat['num_syls'] != df['syls'][idx]:
        hard_words['words'].append(df['words'][idx])
        hard_words['syl_seq'].append(df['syl_seq'][idx])
        hard_words['syls'].append(df['syls'][idx])
        print(df['words'][idx], df['syl_seq'][idx], df['syls'][idx], yhat['num_syls'], yhat['confidence'])
        print(yhat['probs'])

bizerse 1010010 3 2 0.38
[('b', 0.89), ('i', 0.11), ('z', 0.18), ('e', 0.82), ('r', 0.11), ('s', 0.11), ('e', 0.1)]
millerand 100001000 2 3 0.28
[('m', 0.89), ('i', 0.11), ('l', 0.1), ('l', 0.78), ('e', 0.22), ('r', 0.11), ('a', 0.89), ('n', 0.1), ('d', 0.1)]
droolier 10000110 3 2 0.31
[('d', 0.89), ('r', 0.11), ('o', 0.1), ('o', 0.1), ('l', 0.22), ('i', 0.78), ('e', 0.11), ('r', 0.1)]
smet 1100 2 1 0.64
[('s', 0.89), ('m', 0.11), ('e', 0.1), ('t', 0.1)]
maninke 1010010 3 2 0.35
[('m', 0.89), ('a', 0.11), ('n', 0.22), ('i', 0.78), ('n', 0.11), ('k', 0.1), ('e', 0.1)]
federalist 1001000000 2 4 0.33
[('f', 0.89), ('e', 0.11), ('d', 0.1), ('e', 0.89), ('r', 0.11), ('a', 0.89), ('l', 0.11), ('i', 0.89), ('s', 0.11), ('t', 0.1)]
cashdrawer 1000100000 2 3 0.33
[('c', 0.89), ('a', 0.11), ('s', 0.1), ('h', 0.1), ('d', 0.89), ('r', 0.11), ('a', 0.1), ('w', 0.1), ('e', 0.89), ('r', 0.1)]
intermarriage 1010010010000 4 5 0.23
[('i', 0.89), ('n', 0.11), ('t', 0.89), ('e', 0.11), ('r', 0.1), ('m', 0

In [ ]:
while True:
    words = input()
    outputs = counter.predict(words, return_sequence = True, pred_threshold = 0.5)
    string = ''.join([ctext(f'{char} ', deg) for char, deg in outputs['probs']])
    printmd(f'{string}&nbsp;= {outputs["num_syls"]} syllables', size = 5)